# Using the OpenAI API with Pixeltable

PixelTable helps users unify data and computation into a table interface. This notebook shows how we can use the OpenAI API via pixeltable.
We will keep all artifacts within the `demos` directory.

In [1]:
import pixeltable as pxt
cl = pxt.Client()
cl.create_dir('demos', ignore_errors=True)
path = 'demos.chat_completions_demo'

2024-01-29 14:50:16,598 INFO pixeltable env.py:188: found database postgresql://postgres:@/pixeltable?host=/run/user/1000/python_PostgresServer/8fd7da6a31
2024-01-29 14:50:16,692 INFO pixeltable env.py:199: connecting to NOS


/home/marcel/pixeltable/pixeltable/exec/expr_eval_node.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2024-01-29 14:50:16.714 | INFO     | nos.server:init:131 - Inference server already running (name=nos-inference-service-gpu, image=<Image: 'autonomi/nos:0.0.9-gpu'>, id=d4255c36b739).
2024-01-29 14:50:16,715 INFO pixeltable env.py:202: waiting for NOS
2024-01-29 14:50:16,727 INFO pixeltable env.py:223: connecting to OpenAI


# Creating a table with inputs

In [2]:
if path in cl.list_tables():
    cl.drop_table(path)

t = cl.create_table(path, {'id': pxt.IntType(), 'input': pxt.StringType()})

In [3]:
# text from https://en.wikipedia.org/wiki/Global_financial_crisis_in_September_2008
wikipedia_text = '''On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.
The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to "settle securities transactions with customers of Lehman and its clearance parties" according to a statement made in a New York City Bankruptcy court filing.[18]
The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.
On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.
The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]'''

sample_inputs = wikipedia_text.split('\n')

In [4]:
# example row inserted, persisted to table.
t.insert([{'id': 0, 'input': sample_inputs[0]}])
t.show()

Inserting rows into table: 0rows [00:00, ?rows/s]

inserted 1 row with 0 errors 


id,input
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers."


# Making OpenAI API calls

Calling OpenAI API endpoints involves constructing a message object, which we express in Pixeltable by adding a new computed column,i.e., a column that represents a computation.

In [5]:
prompt = "For the following sentence, extract all company names from the text."

msgs = [
    { "role": "system", "content": prompt },
    { "role": "user", "content": t.input }
]

t.add_column(input_msgs=msgs)

Computing cells:   0%|          | 0/1 [00:00<?, ?cells/s]

added 1 column values with 0 errors


UpdateStatus(num_rows=1, num_computed_values=1, num_excs=0, updated_cols=[], cols_with_excs=[])

Unlike the values of the`input` column, which users provide, the `t.input_msgs` column is computed automatically from the `t.input` column values:

In [6]:
t.show()

id,input,input_msgs
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]"


In Pixeltable, OpenAI API calls are exposed as Pixeltable functions, which can be used to create computed columns.

For the following command to work, you must have set `os.environ['OPENAI_API_KEY']`.

In [7]:
from pixeltable.functions.openai import chat_completions
t['chat_output'] = chat_completions(model='gpt-3.5-turbo', messages=t.input_msgs)
t.show()

Computing cells:   0%|          | 0/1 [00:00<?, ?cells/s]

added 1 column values with 0 errors


id,input,input_msgs,chat_output
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]","{'id': 'chatcmpl-8mUkDEuZVPzk6k4veHCYEHSrpLvTM', 'model': 'gpt-3.5-turbo-0613', 'usage': {'total_tokens': 65, 'prompt_tokens': 61, 'completion_tokens': 4}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Lehman Brothers', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1706568617, 'system_fingerprint': None}"


The result objects of the OpenAI API calls are generally complex JSON structures, which require some navigation to extract the response. We can express this as JSON path expressions and create another computed column:

In [8]:
t['response'] = t.chat_output.choices[0].message.content
t.show()

Computing cells:   0%|          | 0/1 [00:00<?, ?cells/s]

added 1 column values with 0 errors


id,input,input_msgs,chat_output,response
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]","{'id': 'chatcmpl-8mUkDEuZVPzk6k4veHCYEHSrpLvTM', 'model': 'gpt-3.5-turbo-0613', 'usage': {'total_tokens': 65, 'prompt_tokens': 61, 'completion_tokens': 4}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Lehman Brothers', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1706568617, 'system_fingerprint': None}",Lehman Brothers


Let's run a query to look only at the input and output:

In [9]:
t.select(t.input, t.response).show()

input,response
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.",Lehman Brothers


Once we have defined these computed columns, much like with a spreadsheet, newly inserted `t.input` values trigger computation of all derived columns, such as `t.response`:

In [10]:
t.insert([{'id': i, 'input': sample_inputs[i]} for i in range(1, len(sample_inputs))])

Computing cells:   0%|          | 0/15 [00:00<?, ?cells/s]

Inserting rows into table: 0rows [00:00, ?rows/s]

inserted 5 rows with 0 errors 


UpdateStatus(num_rows=5, num_computed_values=15, num_excs=0, updated_cols=[], cols_with_excs=[])

In [11]:
t.select(t.input, t.response).show()

input,response
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.",Lehman Brothers
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.,Lehman Brothers
"The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to ""settle securities transactions with customers of Lehman and its clearance parties"" according to a statement made in a New York City Bankruptcy court filing.[18]","Lehman Brothers, JPMorgan Chase"
"The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.","Merrill Lynch, Bank of America"
"On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.",American International Group (AIG)
"The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]","AIG, Federal Reserve"


# Adding ground truth data

Computed table columns are stored and persisted, avoiding repeated computation.
Pixeltable can be used to conduct experiments, for example to compare prompt variations:

We'll start by creating our ground-truth data manually:

In [12]:
t['ground_truth'] = pxt.StringType(nullable=True)

ground_truth = [
    'Lehman Brothers',
    'Lehman Brothers',
    'JP Morgan Chase, Lehman Brothers',
    'Merill Lynch, Bank of America',
    'American International Group',
    'American International Group',
]

for i, gt in enumerate(ground_truth):
    t.update({'ground_truth': gt}, where=(t['id'] == i))

Inserting rows into table: 0rows [00:00, ?rows/s]

Inserting rows into table: 0rows [00:00, ?rows/s]

Inserting rows into table: 0rows [00:00, ?rows/s]

Inserting rows into table: 0rows [00:00, ?rows/s]

Inserting rows into table: 0rows [00:00, ?rows/s]

Inserting rows into table: 0rows [00:00, ?rows/s]

And this is what we have so far:

In [13]:
t.select(t.input, t.response, t.ground_truth).show()

input,response,ground_truth
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.",Lehman Brothers,Lehman Brothers
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.,Lehman Brothers,Lehman Brothers
"The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to ""settle securities transactions with customers of Lehman and its clearance parties"" according to a statement made in a New York City Bankruptcy court filing.[18]","Lehman Brothers, JPMorgan Chase","JP Morgan Chase, Lehman Brothers"
"The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.","Merrill Lynch, Bank of America","Merill Lynch, Bank of America"
"On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.",American International Group (AIG),American International Group
"The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]","AIG, Federal Reserve",American International Group


# Evaluation

Now that we have some ground truth available, we can carry out basic evaluations of the GPT outputs, in this case by asking ChatGPT to decide whether the two are equivalent. 

To start with, we'll create an evaluation prompt:

In [14]:
eval_prompt = '''
Compare the following listA and listB of entities, check if they contains the same entities.
Return a json object with the following format:
{"reasoning": explaining your reasoning, "decision": 1 if the lists matched, 0 otherwise}
'''

t.add_column(
    eval_prompt=[
        { "role": "system", "content": eval_prompt },
        {
            "role": "user",
            "content": pxt.functions.str_format(
                'listA: "{0}" \n listB: "{1}"', t.response, t.ground_truth)
        }])

Computing cells:   0%|          | 0/6 [00:00<?, ?cells/s]

added 6 column values with 0 errors


UpdateStatus(num_rows=6, num_computed_values=6, num_excs=0, updated_cols=[], cols_with_excs=[])

The function `str_format()` is similar to an f-string, but allows values to come from any table column.

In [15]:
t.select(t.eval_prompt).show()

eval_prompt
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, check if they contains the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""Lehman Brothers"" listB: ""Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, check if they contains the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""Lehman Brothers"" listB: ""Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, check if they contains the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""Lehman Brothers, JPMorgan Chase"" listB: ""JP Morgan Chase, Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, check if they contains the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""Merrill Lynch, Bank of America"" listB: ""Merill Lynch, Bank of America""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, check if they contains the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""American International Group (AIG)"" listB: ""American International Group""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, check if they contains the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""AIG, Federal Reserve"" listB: ""American International Group""'}]"


The actual evaluation happens in another computed column:

In [16]:
t['eval'] = chat_completions(model='gpt-3.5-turbo', messages=t.eval_prompt)
t['eval_output'] = t.eval.choices[0].message.content

Computing cells:   0%|          | 0/6 [00:00<?, ?cells/s]

added 6 column values with 0 errors


Computing cells:   0%|          | 0/6 [00:00<?, ?cells/s]

added 6 column values with 0 errors


Let's take a look:

In [17]:
t.select(t.eval, t.eval_output).show()

eval,eval_output
"{'id': 'chatcmpl-8mUkH5nobeETZqebvgcqY0X8N5CiX', 'model': 'gpt-3.5-turbo-0613', 'usage': {'total_tokens': 111, 'prompt_tokens': 81, 'completion_tokens': 30}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{""reasoning"": ""The entities in listA and listB are the same: \'Lehman Brothers\'."", ""decision"": 1}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1706568621, 'system_fingerprint': None}","{""reasoning"": ""The entities in listA and listB are the same: 'Lehman Brothers'."", ""decision"": 1}"
"{'id': 'chatcmpl-8mUkJkp2BuXPxOHLCFeqMLU8R5WCL', 'model': 'gpt-3.5-turbo-0613', 'usage': {'total_tokens': 108, 'prompt_tokens': 81, 'completion_tokens': 27}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{""reasoning"": ""The reasoning is that both lists have the same entity \'Lehman Brothers\'"", ""decision"": 1}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1706568623, 'system_fingerprint': None}","{""reasoning"": ""The reasoning is that both lists have the same entity 'Lehman Brothers'"", ""decision"": 1}"
"{'id': 'chatcmpl-8mUkLgv9bFjgw9fOqOkjTxtelPzVX', 'model': 'gpt-3.5-turbo-0613', 'usage': {'total_tokens': 128, 'prompt_tokens': 89, 'completion_tokens': 39}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{""reasoning"": ""The order of the entities in the lists does not matter. Therefore, we can simply compare the entities in the lists without considering their order."", ""decision"": 1}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1706568625, 'system_fingerprint': None}","{""reasoning"": ""The order of the entities in the lists does not matter. Therefore, we can simply compare the entities in the lists without considering their order."", ""decision"": 1}"
"{'id': 'chatcmpl-8mUkM8A1x7oBowCe9svTy06CnB9iP', 'model': 'gpt-3.5-turbo-0613', 'usage': {'total_tokens': 175, 'prompt_tokens': 88, 'completion_tokens': 87}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{""reasoning"": ""Based on the given lists, both listA and listB contain the same entities, which are \'Merrill Lynch\' and \'Bank of America\'. The only difference is a minor spelling mistake in listB where \'Merrill\' is misspelled as \'Merill\'. Since the entities are the same despite the spelling difference, the lists can be considered as matched."", ""decision"": 1}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1706568626, 'system_fingerprint': None}","{""reasoning"": ""Based on the given lists, both listA and listB contain the same entities, which are 'Merrill Lynch' and 'Bank of America'. The only difference is a minor spelling mistake in listB where 'Merrill' is misspelled as 'Merill'. Since the entities are the same despite the spelling difference, the lists can be considered as matched."", ""decision"": 1}"
"{'id': 'chatcmpl-8mUkQgLIAZBHMFQuCNwyyWbg3A4nb', 'model': 'gpt-3.5-turbo-0613', 'usage': {'total_tokens': 144, 'prompt_tokens': 82, 'completion_tokens': 62}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{""reasoning"": ""The two lists contain the same entity, \'American International Group\'. The only difference is that listA includes the abbreviation \'AIG\' while listB does not. However, since the core entity is the same, we can consider them as a match."", ""decision"": 1}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1706568630, 'system_fingerprint': None}","{""reasoning"": ""The two lists contain the same entity, 'American International Group'. The only difference is that listA includes the abbreviation 